# Read data from bigquery - GIS feature engineering


Historic data coming from ...

* create query to clean data and keep only columns I need
* create bigquery dataset within `event-driven-ml` project

### 2019

In [ ]:
%%bash

bq query \
--destination_table event-driven-ml:edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2019 \
--location=US \
--replace \
--use_legacy_sql=false \
--allow_large_results \
'SELECT 
    GENERATE_UUID() as uuid, 
    pickup_datetime, dropoff_datetime, dayofweek, hourofday, weekofyear, year, 
    pickup_zone_name, dropoff_zone_name, pu_borough, do_borough, pu_zone_geom, do_zone_geom, ST_CENTROID(pu_zone_geom) as pickup_point, ST_CENTROID(do_zone_geom) as dropoff_point, 
    ROUND(ST_DISTANCE(ST_CENTROID(pu_zone_geom), ST_CENTROID(do_zone_geom))) as centroide_distance, trip_distance, speed, trip_duration, 
    passenger_count, flag_tolls, total_amount,
    CASE WHEN pickup_zone_name LIKE "%irport%" OR dropoff_zone_name LIKE "%irport%" THEN 1 ELSE 0 END AS flag_airport,
    CASE WHEN passenger_count <= 4 THEN 1 ELSE 0 END AS flag_5_seat_car
FROM
(SELECT
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) as pickup_datetime,
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) as dropoff_datetime,
    EXTRACT(DAYOFWEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS dayofweek,
    EXTRACT(HOUR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS hourofday,
    EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS weekofyear,
    EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS year,
    DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) AS trip_duration,
    passenger_count,
    CAST(trip_distance AS FLOAT64) AS trip_distance,
    CAST(PULocationID AS INT64) AS pickup_location_id,
    CAST(DOLocationID AS INT64) AS dropoff_location_id,
    CAST(RatecodeID AS INT64) AS rate_code,
    CAST(payment_type AS INT64) AS payment_type,
    CAST(fare_amount AS FLOAT64) AS fare_amount,
    CAST(tolls_amount AS FLOAT64) AS tolls_amount,
    CASE WHEN tolls_amount > 0.0 THEN 1.0 ELSE 0.0 END AS flag_tolls,
    CAST(total_amount AS FLOAT64) AS total_amount,
    CAST(trip_distance AS FLOAT64)/DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) as speed

FROM `event-driven-ml.edml_nyc_yellow_taxi_us.nyc_yellow_taxi_trips_2019`

WHERE
    trip_distance > 0.0
    AND trip_distance < 3000.0
    AND fare_amount >= 2.5
    AND fare_amount < 6000.0
    AND total_amount > 0.0
    AND total_amount < 6000.0
    AND passenger_count > 0
    AND passenger_count <= 9
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) > 0
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) <= 180
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 2018 AND 2019
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 2019 AND 2020

) AS trips

INNER JOIN
(SELECT CAST(zone_id AS INT64) AS pickup_zone_id, zone_name AS pickup_zone_name, zone_geom AS pu_zone_geom, borough AS pu_borough FROM 
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS pu_zones
ON trips.pickup_location_id = pu_zones.pickup_zone_id

INNER JOIN (SELECT CAST(zone_id AS INT64) AS dropoff_zone_id, zone_name AS dropoff_zone_name, zone_geom AS do_zone_geom, borough AS do_borough FROM
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS do_zones
ON trips.dropoff_location_id = do_zones.dropoff_zone_id'

### 2018

In [ ]:
%%bash

bq query \
--destination_table event-driven-ml:edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2018 \
--location=US \
--replace \
--use_legacy_sql=false \
--allow_large_results \
'SELECT 
    GENERATE_UUID() as uuid, 
    pickup_datetime, dropoff_datetime, dayofweek, hourofday, weekofyear, year, 
    pickup_zone_name, dropoff_zone_name, pu_borough, do_borough, pu_zone_geom, do_zone_geom, ST_CENTROID(pu_zone_geom) as pickup_point, ST_CENTROID(do_zone_geom) as dropoff_point, 
    ROUND(ST_DISTANCE(ST_CENTROID(pu_zone_geom), ST_CENTROID(do_zone_geom))) as centroide_distance, trip_distance, speed, trip_duration, 
    passenger_count, flag_tolls, total_amount,
    CASE WHEN pickup_zone_name LIKE "%irport%" OR dropoff_zone_name LIKE "%irport%" THEN 1 ELSE 0 END AS flag_airport,
    CASE WHEN passenger_count <= 4 THEN 1 ELSE 0 END AS flag_5_seat_car
FROM
(SELECT
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) as pickup_datetime,
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) as dropoff_datetime,
    EXTRACT(DAYOFWEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS dayofweek,
    EXTRACT(HOUR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS hourofday,
    EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS weekofyear,
    EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS year,
    DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) AS trip_duration,
    passenger_count,
    CAST(trip_distance AS FLOAT64) AS trip_distance,
    CAST(PULocationID AS INT64) AS pickup_location_id,
    CAST(DOLocationID AS INT64) AS dropoff_location_id,
    CAST(RatecodeID AS INT64) AS rate_code,
    CAST(payment_type AS INT64) AS payment_type,
    CAST(fare_amount AS FLOAT64) AS fare_amount,
    CAST(tolls_amount AS FLOAT64) AS tolls_amount,
    CASE WHEN tolls_amount > 0.0 THEN 1.0 ELSE 0.0 END AS flag_tolls,
    CAST(total_amount AS FLOAT64) AS total_amount,
    CAST(trip_distance AS FLOAT64)/DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) as speed

FROM `event-driven-ml.edml_nyc_yellow_taxi_us.nyc_yellow_taxi_trips_2018`

WHERE
    trip_distance > 0.0
    AND trip_distance < 3000.0
    AND fare_amount >= 2.5
    AND fare_amount < 6000.0
    AND total_amount > 0.0
    AND total_amount < 6000.0
    AND passenger_count > 0
    AND passenger_count <= 9
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) > 0
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) <= 180
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 2017 AND 2018
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 2018 AND 2019

) AS trips

INNER JOIN
(SELECT CAST(zone_id AS INT64) AS pickup_zone_id, zone_name AS pickup_zone_name, zone_geom AS pu_zone_geom, borough AS pu_borough FROM 
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS pu_zones
ON trips.pickup_location_id = pu_zones.pickup_zone_id

INNER JOIN (SELECT CAST(zone_id AS INT64) AS dropoff_zone_id, zone_name AS dropoff_zone_name, zone_geom AS do_zone_geom, borough AS do_borough FROM
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS do_zones
ON trips.dropoff_location_id = do_zones.dropoff_zone_id'

### 2017

In [ ]:
%%bash

bq query \
--destination_table event-driven-ml:edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2017 \
--location=US \
--replace \
--use_legacy_sql=false \
--allow_large_results \
'SELECT 
    GENERATE_UUID() as uuid, 
    pickup_datetime, dropoff_datetime, dayofweek, hourofday, weekofyear, year, 
    pickup_zone_name, dropoff_zone_name, pu_borough, do_borough, pu_zone_geom, do_zone_geom, ST_CENTROID(pu_zone_geom) as pickup_point, ST_CENTROID(do_zone_geom) as dropoff_point, 
    ROUND(ST_DISTANCE(ST_CENTROID(pu_zone_geom), ST_CENTROID(do_zone_geom))) as centroide_distance, trip_distance, speed, trip_duration, 
    passenger_count, flag_tolls, total_amount,
    CASE WHEN pickup_zone_name LIKE "%irport%" OR dropoff_zone_name LIKE "%irport%" THEN 1 ELSE 0 END AS flag_airport,
    CASE WHEN passenger_count <= 4 THEN 1 ELSE 0 END AS flag_5_seat_car
FROM
(SELECT
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) as pickup_datetime,
    DATETIME(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) as dropoff_datetime,
    EXTRACT(DAYOFWEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS dayofweek,
    EXTRACT(HOUR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS hourofday,
    EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) -1 AS weekofyear,
    EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) AS year,
    DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) AS trip_duration,
    passenger_count,
    CAST(trip_distance AS FLOAT64) AS trip_distance,
    CAST(PULocationID AS INT64) AS pickup_location_id,
    CAST(DOLocationID AS INT64) AS dropoff_location_id,
    CAST(RatecodeID AS INT64) AS rate_code,
    CAST(payment_type AS INT64) AS payment_type,
    CAST(fare_amount AS FLOAT64) AS fare_amount,
    CAST(tolls_amount AS FLOAT64) AS tolls_amount,
    CASE WHEN tolls_amount > 0.0 THEN 1.0 ELSE 0.0 END AS flag_tolls,
    CAST(total_amount AS FLOAT64) AS total_amount,
    CAST(trip_distance AS FLOAT64)/DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) as speed

FROM `event-driven-ml.edml_nyc_yellow_taxi_us.nyc_yellow_taxi_trips_2017`

WHERE
    trip_distance > 0.0
    AND trip_distance < 3000.0
    AND fare_amount >= 2.5
    AND fare_amount < 6000.0
    AND total_amount > 0.0
    AND total_amount < 6000.0
    AND passenger_count > 0
    AND passenger_count <= 9
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) > 0
    AND DATETIME_DIFF(CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime) AS DATETIME), 
                  CAST(PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime) AS DATETIME), 
                  MINUTE) <= 180
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(WEEK FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 1 AND 53
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_pickup_datetime)) BETWEEN 2016 AND 2017
    AND EXTRACT(YEAR FROM PARSE_TIMESTAMP("%m/%d/%Y %I:%M:%S %p", tpep_dropoff_datetime)) BETWEEN 2017 AND 2018

) AS trips

INNER JOIN
(SELECT CAST(zone_id AS INT64) AS pickup_zone_id, zone_name AS pickup_zone_name, zone_geom AS pu_zone_geom, borough AS pu_borough FROM 
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS pu_zones
ON trips.pickup_location_id = pu_zones.pickup_zone_id

INNER JOIN (SELECT CAST(zone_id AS INT64) AS dropoff_zone_id, zone_name AS dropoff_zone_name, zone_geom AS do_zone_geom, borough AS do_borough FROM
 `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`) AS do_zones
ON trips.dropoff_location_id = do_zones.dropoff_zone_id'

### Union

#### Train

In [ ]:
%%bash

bq query \
--destination_table event-driven-ml:edml_nyc_yellow_taxi_us.viz_gis_feat_eng_train \
--location=US \
--replace \
--use_legacy_sql=false \
--allow_large_results \
'SELECT * FROM `event-driven-ml.edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2018`
    UNION ALL SELECT * FROM `event-driven-ml.edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2017`'

#### Test

In [ ]:
%%bash

bq query \
--destination_table event-driven-ml:edml_nyc_yellow_taxi_us.viz_gis_feat_eng_test \
--location=US \
--replace \
--use_legacy_sql=false \
--allow_large_results \
'SELECT * FROM `event-driven-ml.edml_nyc_yellow_taxi_us.viz_gis_feat_eng_2019`'